## Pyalgo Tutorial

In [1]:
import sys
paths = ['../scripts',
        '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/quickpipeline',
        '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/pd_feature_union']

for path in paths:
    sys.path.append(path)

from market_imports import *
from market_code import *
from market_trans import *
from quickpipe_mod import * 
from pandas_feature_union import *

data_path = '/Users/jacob/Desktop/docs/kaggle/two_sigma/_g_data/data'
df_market = pd.read_csv(data_path + '/marketdata_sample.csv') 
df_news = pd.read_csv(data_path + '/news_sample.csv')


import warnings
warnings.filterwarnings(action='ignore')

import datetime as dt



In [2]:
# TRYING TO PREVENT DATE TYPE CONVERSION
# THIS SHOULD BE UPDATED FROM PANDA DATAREADER
csv = pd.read_csv(data_path + '/sigma_data.csv', index_col='Date', parse_dates=True, 
                 infer_datetime_format=True)

csv.shape

(57384, 9)

In [3]:
# COMPUTE RESIDUAL FOR RESPONSE
import statsmodels.tsa.api as tsa
from statsmodels.tsa.seasonal import seasonal_decompose 

x = csv[['returns_open_raw10_next']]
# Freq will depend on asset number - this needs to be researched
result = seasonal_decompose(x, model='additive', freq=50)

csv['returnsOpenNextMktres10'] = result.resid
# Lose the first and last day
csv = csv[-csv['returnsOpenNextMktres10'].isnull()]
csv.drop(['returns_open_raw10_next'], axis=1, inplace=True)

check = csv.columns[:4].tolist()
csv.drop_duplicates(check, inplace=True)

# ELC is insanely high. The next highest price is around 300
csv[csv.Close ==csv.Close.max()]
csv[csv.Close > 1000]
csv = csv[csv.asset != 'ELC']

# Matching the kernel data some more
csv.index.name = 'time'
csv.reset_index(inplace=True)
csv.rename(columns={'asset': 'assetCode', 'Volume': 'volume',
                  'Close': 'close'}, inplace=True)
csv.head()

,time,assetCode,Open,close,volume,returns_close_raw,returns_open_raw,returns_close_raw10,returns_open_raw10,returnsOpenNextMktres10
0,2014-01-16,QCOM,74.470001,74.720001,10263200.0,0.002814,0.013519,0.018914,0.011615,-0.015252
1,2014-01-16,KEP,16.660000,16.719999,205300.0,0.009012,0.000000,0.017497,0.009650,-0.010187
2,2014-01-16,NWS,17.059999,17.040001,506000.0,-0.006435,-0.011076,-0.048678,-0.044707,0.050676
3,2014-01-16,BHP,66.570000,66.889999,5735900.0,0.029432,0.033296,-0.001494,-0.012391,0.013469
4,2014-01-16,NGG,70.043671,69.934494,212800.0,0.001875,0.004530,-0.004361,-0.009308,-0.042414
